<a href="https://colab.research.google.com/github/ParkSomin23/Voice_EmotionDetection/blob/master/EMO_2_2_Data_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import preprocessing
from keras.utils import np_utils, to_categorical
from sklearn.preprocessing import LabelEncoder
import pickle

import librosa
import librosa.display
import IPython.display as ipd

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram

import pandas as pd
import glob
from sklearn.metrics import confusion_matrix

import os
import sys

from tqdm import tqdm, tqdm_pandas

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
def prepare_data(df, n, aug, mfcc):
    X = np.empty(shape=(df.shape[0], n, 216, 1))
    input_length = sampling_rate * audio_duration
    
    cnt = 0
    for fname in tqdm(df.path):

        file_path = fname
        data, _ = librosa.load(file_path, sr=sampling_rate
                               ,res_type="kaiser_fast"
                               ,duration=2.5
                               ,offset=0.5
                              )

        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, int(input_length) - len(data) - offset), "constant")

        # Augmentation? 
        if aug == 1:
            data = speedNpitch(data)
        
        # which feature?
        if mfcc == 1:
            # MFCC extraction 
            MFCC = librosa.feature.mfcc(data, sr=sampling_rate, n_mfcc=n_mfcc) # (n_mfcc, time?)
            MFCC = np.expand_dims(MFCC, axis=-1) # (n_mfcc, time?, 1)
            X[cnt,] = MFCC # (file_num, n_mfcc, time?, 1)
            
        else:
            # Log-melspectogram
            melspec = librosa.feature.melspectrogram(data, n_mels = n_melspec)   
            logspec = librosa.amplitude_to_db(melspec)
            logspec = np.expand_dims(logspec, axis=-1)
            X[cnt,] = logspec
            
        cnt += 1

        #print(X.shape)
    
    return X

In [ ]:
ref = pd.read_csv("/gdrive/My Drive/dataset_3.csv")
ref

,Unnamed: 0,labels,source,path
0,0,male_neutral,SAVEE,/gdrive/My Drive/SAVEE/DC_n11.wav
1,1,male_angry,SAVEE,/gdrive/My Drive/SAVEE/DC_a12.wav
2,2,male_fear,SAVEE,/gdrive/My Drive/SAVEE/JE_f04.wav
3,3,male_disgust,SAVEE,/gdrive/My Drive/SAVEE/JK_d15.wav
4,4,male_happy,SAVEE,/gdrive/My Drive/SAVEE/DC_h14.wav
...,...,...,...,...
4519,4519,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
4520,4520,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
4521,4521,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...
4522,4522,female_sad,TESS,/gdrive/My Drive/TESS/TESS Toronto emotional s...


In [ ]:
sampling_rate= 44100
audio_duration=2.5
n_mfcc = 30
mfcc = prepare_data(ref, n = n_mfcc, aug = 0, mfcc = 1)

import pickle
with open("/gdrive/My Drive/mfcc.pkl", 'wb') as f:
    pickle.dump(mfcc, f)

100%|██████████| 4524/4524 [31:32<00:00,  2.39it/s]


In [ ]:
c_ref = pd.read_csv("/gdrive/My Drive/dataset_crema_2.csv")
c = c_ref.drop(7062)
c

,Unnamed: 0,labels,source,path
0,4524,male_angry,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_ANG_XX.wav
1,4525,male_disgust,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_DIS_XX.wav
2,4526,male_fear,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_FEA_XX.wav
3,4527,male_happy,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_HAP_XX.wav
4,4528,male_neutral,CREMA,/gdrive/My Drive/CREMA-D/1001_DFA_NEU_XX.wav
...,...,...,...,...
7057,11581,male_fear,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_FEA_XX.wav
7058,11582,male_happy,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_HAP_XX.wav
7059,11583,male_neutral,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_NEU_XX.wav
7060,11584,male_sad,CREMA,/gdrive/My Drive/CREMA-D/1087_IOM_SAD_XX.wav


In [ ]:
sampling_rate= 44100
audio_duration=2.5
n_mfcc = 30
mfcc_c = prepare_data(c, n = n_mfcc, aug = 0, mfcc = 1)

import pickle
with open("/gdrive/My Drive/mfcc_c_2.pkl", 'wb') as f:
    pickle.dump(mfcc_c, f)

100%|██████████| 7062/7062 [35:21<00:00,  3.33it/s]


In [ ]:
with open("/gdrive/My Drive/mfcc_c_2.pkl", 'rb') as f:
    d = pickle.load(f)

d.shape

(7062, 30, 216, 1)

# MEL

In [ ]:
ref = pd.read_csv("/gdrive/My Drive/dataset_3.csv")
sampling_rate=44100
audio_duration=2.5
n_melspec = 30
mel = prepare_data(c, n = n_melspec, aug = 0, mfcc = 0)

import pickle
with open("/gdrive/My Drive/mel_crema.pkl", 'wb') as f:
    pickle.dump(mel, f) #crema

100%|██████████| 7062/7062 [24:22<00:00,  4.83it/s]


In [ ]:
with open("/gdrive/My Drive/mel_crema.pkl", 'rb') as f:
    d = pickle.load(f)

d.shape

(7062, 30, 216, 1)

In [ ]:
sampling_rate=44100
audio_duration=2.5
n_melspec = 30
c_mel = prepare_data(ref, n = n_melspec, aug = 0, mfcc = 0)

import pickle
with open("/gdrive/My Drive/dataset_mel.pkl", 'wb') as f:
    pickle.dump(c_mel, f) #  savee

100%|██████████| 4524/4524 [17:47<00:00,  4.24it/s]
